# 1.Detección de manos con MediaPipe


MediaPipe es excelente para la detección y rastreo de manos en tiempo real, generando puntos clave (landmarks) que se usarán para entrenar el modelo.

In [6]:
import mediapipe as mp
print(mp.__version__)

ModuleNotFoundError: No module named 'mediapipe'

In [5]:
import cv2
import mediapipe as mp

# Inicializa MediaPipe para manos
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# Captura desde la cámara
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir a RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Detección de Manos", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Salir con 'ESC'
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'mediapipe'

Este código detecta las manos en tiempo real y dibuja sus puntos clave (landmarks). Estos puntos se utilizarán como entrada para el modelo de reconocimiento de señas.

# 2.Recolección de datos para entrenamiento


Para traducir lengua de señas, necesitas capturar datos de gestos específicos asociados a cada letra o palabra.

Pasos:
Diseña una estructura de carpetas:

dataset/A para la letra "A".

dataset/B para la letra "B".

Repite para todas las letras o palabras de interés.

Script para capturar datos: Usa los puntos clave detectados por MediaPipe para guardar los datos como características.



In [ ]:
import os
import numpy as np

# Directorio para guardar datos
dataset_dir = "dataset"
os.makedirs(dataset_dir, exist_ok=True)

gesture = "A"  # Cambia según el gesto
save_dir = os.path.join(dataset_dir, gesture)
os.makedirs(save_dir, exist_ok=True)

cap = cv2.VideoCapture(0)
counter = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Extrae puntos clave
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])

            # Guarda como archivo .npy
            landmarks = np.array(landmarks).flatten()
            np.save(os.path.join(save_dir, f"frame_{counter}.npy"), landmarks)
            counter += 1

    cv2.imshow("Recolección de Datos", frame)
    if cv2.waitKey(1) & 0xFF == 27 or counter >= 100:  # Salir con 'ESC'
        break

cap.release()
cv2.destroyAllWindows()


Este script guarda los puntos clave como archivos .npy, que serán usados como entradas para el modelo de aprendizaje.



# 4. Entrenamiento del modelo

Utiliza los datos recolectados para entrenar un modelo de clasificación

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Carga los datos
def load_data(dataset_dir):
    data = []
    labels = []
    for label, gesture in enumerate(os.listdir(dataset_dir)):
        gesture_dir = os.path.join(dataset_dir, gesture)
        for file in os.listdir(gesture_dir):
            filepath = os.path.join(gesture_dir, file)
            landmarks = np.load(filepath)
            data.append(landmarks)
            labels.append(label)
    return np.array(data), np.array(labels)

X, y = load_data("dataset")
y = tf.keras.utils.to_categorical(y, num_classes=len(os.listdir("dataset")))

# Define el modelo
model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(len(os.listdir("dataset")), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrena el modelo
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)
model.save("gesture_model.h5")


# 5. Predicción y traducción en tiempo real

Carga el modelo entrenado y usa los puntos clave en tiempo real para hacer predicciones.

In [ ]:
model = tf.keras.models.load_model("gesture_model.h5")

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])
            landmarks = np.array(landmarks).flatten()
            landmarks = np.expand_dims(landmarks, axis=0)

            prediction = model.predict(landmarks)
            gesture = np.argmax(prediction)
            print(f"Gesto detectado: {gesture}")

    cv2.imshow("Traducción de Señales", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Salir con 'ESC'
        break

cap.release()
cv2.destroyAllWindows()
